# Required Libraries and Modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ref_curve_task2 import reference_curve
import dynamics as dyn
from Reg_Newton_Method import Newton_reg
from LQR import lqr_affine
from dynamics import dt
from LQR_tracking import lqr_track
from animation import animation
from plots import traj_gen_plot,traj_track_plot,track_error_plot
from mpc import mpc_controller1,mpc_controller2



# Allow Ctrl-C to work despite plotting
import signal

signal.signal(signal.SIGINT, signal.SIG_DFL)

#######################################
# Parameters from dynamics
#######################################

# Number of states
ns = dyn.ns
# Number of inputs
ni = dyn.ni

# Reference curve for trajectory generation

In [2]:
######################################################
# Reference curve and number of discrete time samples
######################################################

xref,uref,T = reference_curve()
xref = xref.T
uref = uref.reshape(1, -1)

# Set the reference angular velocities constant equal to 0
xref[2,:]=0
xref[3,:]=0


# Trajectory generation with Newton's method

In [ ]:
#######################################
# Newton algorithm parameters
#######################################

# Maximum number of iterations for Newton's method
maxiters = 100
fixed_stepsize = 1 #(not used)

# Armijo parameters
stepsize_0 = 1
c = 0.5
beta = 0.7
armijo_maxiters = 20 
term_cond = 1e-5 

#############################################################
# Weight matrices for cost function for trajectory generation
#############################################################

Qt = np.diag([100000,100000,0.01,0.01])
Rt = 100*np.eye(ni)
QT = Qt

######################################
# Initial guess for the Newton method
######################################

xinit = np.repeat(xref[:,0].reshape(-1,1), T, axis=1)
uinit = np.repeat(uref[:,0].reshape(-1,1), T, axis=1)

######################################
# Newton's method
######################################

# Flag used to show or not plots in Newton_reg function (cost and descent direction norm)
show = False
# Flag used to show trajectories at each iteration of Newton's method
traj_show = False
# Flag used to show or not the descent armijo plot
descent_plot = False

x,u,maxiters = Newton_reg(xref,uref,xinit,uinit,T,dt,maxiters,Qt,Rt,QT,
                          stepsize_0,beta,c,armijo_maxiters,show,traj_show,descent_plot,term_cond=term_cond)

# Optimal trajectory
xstar = x[:,:,maxiters-1]
ustar = u[:,:,maxiters-1]

# Plots

In [4]:
ustar[:,-1] = ustar[:,-2] # for plotting purposes

# Final time
tf = T*dt

t_hor = np.linspace(0, tf, T)

# Trajectory generation plots

In [ ]:
######################################
# Trajectory generation plots
######################################

# Optimal trajectory and reference curve

traj_gen_plot(t_hor,xstar,xref,ustar,uref,ns)

# Trajectory tracking with LQR 

In [8]:
#######################################
# Trajectory tracking with LQR (Task3)
#######################################

# weight matrices for the regulator
Qreg = np.diag([10000,10000,100,100])
Rreg = 1*np.eye(ni)
QTreg = Qreg

# Nominal x0 of the reference trajectory 
x0 = np.array([xstar[0,0],xstar[1,0],xstar[2,0],xstar[3,0]])
# Noise
#noise = np.array([0.3,-0.4, 0, 0])
noise = np.array([0.5,-0.6, 0.2, 0.2])
# Perturbed initial condition
x0 = x0 + noise

# Real trajectory followed by the robot with lqr
xreal_lqr,ureal_lqr = lqr_track(xstar,ustar,T,Qreg,Rreg,QTreg,x0)

# Trajectory tracking with LQR plots

In [ ]:
######################################
# Trajectory tracking with LQR plots
######################################
ureal_lqr[:,-1] = ureal_lqr[:,-2] # for plotting purposes

# Trajectory tracking plots
traj_track_plot(t_hor,xreal_lqr,xstar,ureal_lqr,ustar,ns)

# Tracking error plots
track_error_plot(t_hor,xreal_lqr,xstar,ureal_lqr,ustar,ns)


# Trajectory tracking with MPC 

In [14]:
#######################################
# Trajectory tracking with mpc (Task4)
#######################################

# weight matrices for the regulator
Qmpc = np.diag([10000,10000,100,100])
Rmpc = 0.001*np.eye(ni)
QTmpc = Qmpc

# Nominal x0 of the reference trajectory 
x0 = np.array([xstar[0,0],xstar[1,0],xstar[2,0],xstar[3,0]])
# Noise
noise = np.array([0.1, -0.1, 0, 0])
# Perturbed initial condition
x0 = x0 + noise
#Prediction horizon
pred_hor = 5
# Flag to choose the desired mpc function
mpc1 = True

# Real trajectory followed by the robot with mpc
if mpc1:
    xreal_mpc,ureal_mpc = mpc_controller1(xstar,ustar,noise,pred_hor,Qmpc,Rmpc,QTmpc)
else:
    xreal_mpc,ureal_mpc = mpc_controller2(Qmpc,Rmpc,QTmpc,xstar,ustar,x0,T,pred_hor)


# Trajectory tracking with MPC plots 

In [ ]:
######################################
# Trajectory tracking with MPC plots
######################################
ureal_mpc[:,-1] = ureal_mpc[:,-2] # for plotting purposes

sim_steps = xreal_mpc.shape[1]
xdes = xstar[:,:sim_steps]
udes = ustar[:,:sim_steps]

# Final time
final_time = sim_steps*dt

time = np.linspace(0, final_time, sim_steps)

# Trajectory tracking plots
traj_track_plot(time,xreal_mpc,xdes,ureal_mpc,udes,ns)

# Tracking error plots
track_error_plot(time,xreal_mpc,xdes,ureal_mpc,udes,ns)


# Animation of the robot 

In [10]:
#########################################
# Animation of the robot executing task 3
#########################################

animation(xreal_lqr,xstar)